In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

pd.options.display.max_rows = 4000
pd.options.display.max_seq_items = 2000

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
path_contracts_lookup = '/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/contractsWithERCFlags.csv'
path_block_times = '/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/blockTimes.csv'

## UtC->UtC - Why is this happening?
We found 46 occurence at day 1523577600000000000 of UtC->UtC. We will examine that next

In [3]:
transitions_543 = pd.read_csv('/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/parity_transactions/mr_2018-04-13 00:00:00_in_transactions5430000-5439999_7_transitions.csv', index_col=0)

/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/distributed_ledger_process_miner/dl-pm/lib/python3.6/site-packages/numpy/lib/arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
ucucs = transitions_543[transitions_543['transition']=='UtC->UtC']


In [5]:
ucucs.head()

,total_pos,timestamp,transition
1108776,5432219000545019,1523608857,UtC->UtC
1108778,5432219000545020,1523608857,UtC->UtC
1122114,5432248000551726,1523609218,UtC->UtC
1122116,5432248000551727,1523609218,UtC->UtC
1135380,5432273000558389,1523609537,UtC->UtC


From only observing the total position tags we can see, that these are coming in pairs, which means that we have a sequence of 3 UtC events directly successing each other. So three UtC in a row?

In [9]:
block_times = pd.read_csv(path_block_times, index_col=0)
transaction_lookup = pd.read_csv('/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/transaction_lookup.csv')

/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/distributed_ledger_process_miner/dl-pm/lib/python3.6/site-packages/numpy/lib/arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
log_543 = pd.read_csv('/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/parity_transactions/ps_transactions5430000-5439999_event_log.csv', index_col=0)

/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/distributed_ledger_process_miner/dl-pm/lib/python3.6/site-packages/numpy/lib/arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [10]:
candidate_tx_hashes_ucuc = log_543.merge(ucucs, on='total_pos').merge(transaction_lookup, left_on='transaction_id', right_on='id')['transaction_hash']
candidate_tx_hash_ucuc_list = candidate_tx_hashes_ucuc.values.tolist()

In [11]:
raw_543 = pd.read_csv('/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/parity_transactions/transactions5430000-5439999.csv')

/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/distributed_ledger_process_miner/dl-pm/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
contracts_lookup = pd.read_csv(path_contracts_lookup, index_col=0)

/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/distributed_ledger_process_miner/dl-pm/lib/python3.6/site-packages/numpy/lib/arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [15]:
res_543 = raw_543[raw_543['transactionHash'].isin(candidate_tx_hash_ucuc_list)].merge(contracts_lookup, left_on='action.from', right_on='result.address', how='left').merge(contracts_lookup, how='left', left_on='action.to', right_on='result.address')
res_543.head(10)

,action.address,action.balance,action.callType,action.from,action.gas,action.init,action.input,action.refundAddress,action.to,action.value,blockHash,blockNumber,error,result,result.address_x,result.code,result.gasUsed,result.output,subtraces,traceAddress,transactionHash,transactionPosition,type,result.address_y,isERC20_x,timestamp_x,result.address,isERC20_y,timestamp_y
0,NaN,NaN,NaN,0xc54100fc034d412c21ba92ccf2d916374ac22555,0x3be480,0x6060604090815260018054600160a060020a03191673...,NaN,NaN,NaN,0x1,0xe72a0839928e064657785e7deb3c3b0349829f8527be...,5432219,NaN,NaN,0x4f34892790a78f56cb0f53e7374e77eea940b666,0x,0x344cf,NaN,4,[],0x4ddde8d8480322ce631440a6e14c77d89d73e3986289...,61.0,create,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,call,0x4f34892790a78f56cb0f53e7374e77eea940b666,0x3a8714,NaN,0xd0e30db0,NaN,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,0x1,0xe72a0839928e064657785e7deb3c3b0349829f8527be...,5432219,NaN,NaN,NaN,NaN,0xdd06,0x,0,[0],0x4ddde8d8480322ce631440a6e14c77d89d73e3986289...,61.0,call,NaN,NaN,NaN,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,False,1.523436e+09
2,NaN,NaN,call,0x4f34892790a78f56cb0f53e7374e77eea940b666,0x39a5f3,NaN,0x0bd18d7a0000000000000000000000004f34892790a7...,NaN,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,0x0,0xe72a0839928e064657785e7deb3c3b0349829f8527be...,5432219,NaN,NaN,NaN,NaN,0x208,0x00000000000000000000000000000000000000000000...,0,[1],0x4ddde8d8480322ce631440a6e14c77d89d73e3986289...,61.0,call,NaN,NaN,NaN,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,False,1.523436e+09
3,NaN,NaN,call,0x4f34892790a78f56cb0f53e7374e77eea940b666,0x399df6,NaN,0x2e1a7d4d000000000000000000000000000000000000...,NaN,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,0x0,0xe72a0839928e064657785e7deb3c3b0349829f8527be...,5432219,NaN,NaN,NaN,NaN,0x1d476,0x,1,[2],0x4ddde8d8480322ce631440a6e14c77d89d73e3986289...,61.0,call,NaN,NaN,NaN,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,False,1.523436e+09
4,NaN,NaN,call,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,0x8fc,NaN,0x,NaN,0x4f34892790a78f56cb0f53e7374e77eea940b666,0x1,0xe72a0839928e064657785e7deb3c3b0349829f8527be...,5432219,NaN,NaN,NaN,NaN,0x0,0x,0,"[2, 0]",0x4ddde8d8480322ce631440a6e14c77d89d73e3986289...,61.0,call,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,False,1.523436e+09,NaN,NaN,NaN
5,0x4f34892790a78f56cb0f53e7374e77eea940b666,0x1,NaN,NaN,NaN,NaN,NaN,0xc54100fc034d412c21ba92ccf2d916374ac22555,NaN,NaN,0xe72a0839928e064657785e7deb3c3b0349829f8527be...,5432219,NaN,NaN,NaN,NaN,NaN,NaN,0,[3],0x4ddde8d8480322ce631440a6e14c77d89d73e3986289...,61.0,suicide,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,0xc54100fc034d412c21ba92ccf2d916374ac22555,0x3be480,0x6060604090815260018054600160a060020a03191673...,NaN,NaN,NaN,0x1,0x5d44d7c377a718e6a862875b1fb91bc445417a47ee6a...,5432248,NaN,NaN,0x7f8b5ac0f737bb8dbb8b4f8a14b0484065f817dd,0x,0x3cb51,NaN,4,[],0x4ceb112f04c2cd53a36101125c12c3fe2674a2310652...,12.0,create,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,call,0x7f8b5ac0f737bb8dbb8b4f8a14b0484065f817dd,0x3a8714,NaN,0xd0e30db0,NaN,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,0x1,0x5d44d7c377a718e6a862875b1fb91bc445417a47ee6a...,5432248,NaN,NaN,NaN,NaN,0xdd06,0x,0,[0],0x4ceb112f04c2cd53a36101125c12c3fe2674a2310652...,12.0,call,NaN,NaN,NaN,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,False,1.523436e+09
8,NaN,NaN,call,0x7f8b5ac0f737bb8dbb8b4f8a14b0484065f817dd,0x39a5f3,NaN,0x0bd18d7a0000000000000000000000007f8b5ac0f737...,NaN,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,0x0,0x5d44d7c377a718e6a862875b1fb91bc445417a47ee6a...,5432248,NaN,NaN,NaN,NaN,0x208,0x00000000000000000000000000000000000000000000...,0,[1],0x4ceb112f04c2cd53a36101125c12c3fe2674a2310652...,12.0,call,NaN,NaN,NaN,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,False,1.523436e+09
9,NaN,NaN,call,0x7f8b5ac0f737bb8dbb8b4f8a14b0484065f817dd,0x399df6,NaN,0x2e1a7d4d000000000000000000000000000000000000...,NaN,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,0x0,0x5d44d7c377a718e6a862875b1fb91bc445417a47ee6a...,5432248,NaN,NaN,NaN,NaN,0x25af8,0x,2,[2],0x4ceb112f04c2cd53a36101125c12c3f

In [24]:
res_543[['action.from', 'action.to', 'result.address_x', 'isERC20_x', 'result.address_y', 'isERC20_x', 'subtraces', 'type', 'result.address', 'transactionHash']]

,action.from,action.to,result.address_x,isERC20_x,result.address_y,isERC20_x,subtraces,type,result.address,transactionHash
0,0xc54100fc034d412c21ba92ccf2d916374ac22555,NaN,0x4f34892790a78f56cb0f53e7374e77eea940b666,NaN,NaN,NaN,4,create,NaN,0x4ddde8d8480322ce631440a6e14c77d89d73e3986289...
1,0x4f34892790a78f56cb0f53e7374e77eea940b666,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,NaN,NaN,NaN,NaN,0,call,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,0x4ddde8d8480322ce631440a6e14c77d89d73e3986289...
2,0x4f34892790a78f56cb0f53e7374e77eea940b666,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,NaN,NaN,NaN,NaN,0,call,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,0x4ddde8d8480322ce631440a6e14c77d89d73e3986289...
3,0x4f34892790a78f56cb0f53e7374e77eea940b666,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,NaN,NaN,NaN,NaN,1,call,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,0x4ddde8d8480322ce631440a6e14c77d89d73e3986289...
4,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,0x4f34892790a78f56cb0f53e7374e77eea940b666,NaN,False,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,False,0,call,NaN,0x4ddde8d8480322ce631440a6e14c77d89d73e3986289...
5,NaN,NaN,NaN,NaN,NaN,NaN,0,suicide,NaN,0x4ddde8d8480322ce631440a6e14c77d89d73e3986289...
6,0xc54100fc034d412c21ba92ccf2d916374ac22555,NaN,0x7f8b5ac0f737bb8dbb8b4f8a14b0484065f817dd,NaN,NaN,NaN,4,create,NaN,0x4ceb112f04c2cd53a36101125c12c3fe2674a2310652...
7,0x7f8b5ac0f737bb8dbb8b4f8a14b0484065f817dd,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,NaN,NaN,NaN,NaN,0,call,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,0x4ceb112f04c2cd53a36101125c12c3fe2674a2310652...
8,0x7f8b5ac0f737bb8dbb8b4f8a14b0484065f817dd,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,NaN,NaN,NaN,NaN,0,call,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,0x4ceb112f04c2cd53a36101125c12c3fe2674a2310652...
9,0x7f8b5ac0f737bb8dbb8b4f8a14b0484065f817dd,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,NaN,NaN,NaN,NaN,2,call,0x425e9ec3181d9ddfbfc417f1b9b55dbbd78c3a05,0x4ceb112f04c2cd53a36101125c12c3fe2674a2310652...


As we can see the UtC->UtC come exactly from that behavior: (result address_x translates to the result address of the contract, result address_y translates to
* Contract creates contract (0)
* Then automatically 3 other contracts get created. 
* They get assigned to their from value as the newly create first contract
* That contract later suicides, so we do not have it in or list of contracts

annomly happens. Are these contract factories?